In [1]:
import heapq as hq

In [2]:
import numpy as np

In [3]:
import random

In [4]:
import itertools

In [5]:
def generateEnv(r, c, p):
    env = np.zeros((r,c))
    indexes = random.sample(np.arange(0, r*c), int(float(p)/100*r*c))
    for num in indexes:
        env[num/r, num%c] = 1
    return env

In [6]:
import Queue as Q

In [7]:
def getSucc(env, x, y, a, b):
    suc = []
    for i in range(len(a)):
        if(x + a[i] >= 0 and x + a[i] < env.shape[0] and y + b[i] >= 0 and y + b[i] < env.shape[1]):
            suc.append((x+a[i], y+b[i]))
    return suc

In [8]:
a = [-1, 1, 0, 0]
b = [0, 0, -1, 1]

In [9]:
def bfs(q, coord, vis):
    while(q.empty() == False):
        (x,y) = q.get()
        if(vis[x,y]):
            continue
        
        coord.append((x,y))
        vis[x,y] = 1

        if(env[x,y] == 1):
            env[x,y] = 0
        if(np.array_equal(env, goal)):
            return (coord, env)
        suc = getSucc(env, x, y, a, b)
        for s in suc:
            q.put(s)

In [10]:
import sys

In [11]:
def dfs(st, coord, vis):
    t1_nodes = 0
    t1_mem = sys.getsizeof((1,2))
    t1_stack = 0
    t1_cost = 0
    
    while(len(st) > 0):
        (x,y) = st.pop()
        if(vis[x,y]): 
            continue
        
        coord.append((x,y))
        t1_nodes += 1
        t1_cost = t1_cost + 2
        vis[x,y] = 1
        if(env[x,y] == 1):
            env[x,y] = 0
            t1_cost = t1_cost + 1
        if(np.array_equal(env, goal)):
            return (coord, env, t1_nodes, t1_mem, t1_stack, t1_cost)
        suc = getSucc(env, x, y, a, b)
        #introducing random choice between up, down, left, right
        #random.shuffle(suc)
        for s in suc:
            st.append(s)
        t1_stack = max(t1_stack, len(st))

In [12]:
def initializations(r,c,p):
    coord = []
    env = generateEnv(r,c,p)
    goal = np.zeros(env.shape)
    vis = np.zeros(env.shape)
    q = Q.Queue()
    q.put((0,0))
    st = []
    st.append((0,0))
    pq = []
    counter = itertools.count()
    hq.heappush(pq, (0, 0, 0))
    env
    return coord, env, goal, vis, q, st, pq, counter

In [13]:
(coord, env, goal, vis, q, st, pq, counter) = initializations(10,10,70)

In [14]:
import timeit

In [15]:
start_time = timeit.default_timer()
(path, final_env, t1_nodes, t1_mem, t1_stack, t1_cost) = dfs(st, coord, vis)
t1_time = timeit.default_timer() - start_time

In [16]:
t1_nodes = len(path)

In [17]:
print path

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 9), (1, 8), (1, 7), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 9), (3, 8), (3, 7), (3, 6), (3, 5), (3, 4), (3, 3), (3, 2), (3, 1), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 9), (5, 8), (5, 7), (5, 6), (5, 5), (5, 4), (5, 3), (5, 2), (5, 1), (5, 0), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (7, 9), (7, 8), (7, 7), (7, 6), (7, 5), (7, 4), (7, 3), (7, 2), (7, 1), (7, 0), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (9, 9), (9, 8), (9, 7), (9, 6), (9, 5), (9, 4), (9, 3), (9, 2), (9, 1), (9, 0)]


In [18]:
t1_mem = sys.getsizeof((1,2))

In [19]:
print t1_nodes
print t1_mem
print t1_stack
print t1_cost
print t1_time

100
72
212
270
0.00171113014221


## Implementing A* search

In [20]:
def A_star1(pq, coord, vis, heuristic):
    t2_nodes = 0
    t2_mem = sys.getsizeof((1,2))
    t2_stack = 0
    t2_cost = 0
    
    while(len(pq) > 0):
        #print "getting here"
        (h, x,y) = hq.heappop(pq)
        
        if(vis[x,y] == 1): 
            continue
        
        t2_nodes += 1
        coord.append((x,y))
        t2_cost += 2
        vis[x,y] += 1
        if(env[x,y] == 1):
            env[x,y] = 0
            t2_cost += 1
        if(np.array_equal(env, goal)):
            return (coord, env, t2_nodes, t2_mem, t2_stack, t2_cost)
        suc = getSucc(env, x, y, a, b)
        #print x, y, h
        if(heuristic == 1):
            H = getHeuristic(suc, env, goal)
        
        elif(heuristic == 2):
            H = getHeuristic2(suc, env, goal, x, y)
        
        for i in range(len(suc)):
            hq.heappush(pq, (H[i], suc[i][0], suc[i][1]))
        
        t2_stack = max(t2_stack, len(pq))

### 1. Heuristic 1: h = number of dirty tiles remaining

### This favours moving towards dirty tiles

In [21]:
def getHeuristic(suc, env, goal):
    H = []
    for s in suc:
        
        h = -1 * env[s]                
        H.append(h)
    return np.array(H)

### Heuristic 2: h = manhattan distance from all dirty tiles in it's window

In [22]:
def getManhattanDist(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

In [23]:
def getHeuristic2(suc, env, goal, x, y):
    H = []
    for s in suc:
        h = 0
        
        for i in range(max(0, x-1), min(x+2, env.shape[0])):
            for j in range(max(0, y-1), min(y+2, env.shape[1])):
                if(env[i][j] == 1):
                    h += getManhattanDist((i,j), s)
        H.append(h)
    return H

In [24]:
import itertools

In [25]:
(coord, env, goal, vis, q, st, pq, counter) = initializations(10,10,70)

In [26]:
start_time = timeit.default_timer()
(path, final_env, t2_nodes, t2_mem, t2_stack, t2_cost) = A_star1(pq, coord, vis, 1)
t2_time = timeit.default_timer() - start_time

In [27]:
len(path)

95

In [28]:
print path

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 0), (1, 1), (1, 3), (1, 4), (1, 5), (1, 6), (1, 8), (1, 9), (2, 0), (2, 1), (2, 2), (2, 6), (2, 7), (2, 8), (2, 9), (3, 1), (3, 2), (3, 6), (3, 5), (3, 9), (4, 1), (4, 0), (4, 2), (4, 3), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 2), (5, 3), (5, 5), (5, 6), (5, 7), (5, 8), (6, 2), (6, 3), (6, 4), (6, 5), (6, 7), (6, 8), (6, 9), (7, 2), (7, 1), (7, 0), (7, 4), (7, 5), (7, 8), (8, 0), (8, 2), (8, 3), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (9, 2), (9, 1), (9, 8), (9, 9), (0, 9), (1, 2), (1, 7), (2, 3), (2, 4), (2, 5), (3, 0), (3, 3), (3, 4), (3, 7), (3, 8), (4, 4), (5, 0), (5, 1), (5, 4), (5, 9), (6, 0), (6, 1), (6, 6), (7, 3), (7, 6), (7, 7), (7, 9), (8, 1), (8, 4), (9, 4)]


In [29]:
print t2_nodes
print t2_mem
print t2_stack
print t2_cost
print t2_time

95
72
162
260
0.00323700904846


## Analysis Module

### Comparitive Analysis

In [30]:
print "mem used in t1 = " + str(t1_nodes*t1_mem)
print "mem used in t2 = " + str(t2_nodes*t2_mem)

mem used in t1 = 7200
mem used in t2 = 6840


In [31]:
cost_t2 = []
for i in range(10):
    
    (coord, env, goal, vis, q, st, pq, counter) = initializations(10,10,70)
    (path, final_env, t2_nodes, t2_mem, t2_stack, t2_cost) = A_star1(pq, coord, vis, 1)
    cost_t2.append(t2_cost)

In [32]:
cost_t1 = []
for i in range(10):
    
    (coord, env, goal, vis, q, st, pq, counter) = initializations(10,10,70)
    (path, final_env, t1_nodes, t1_mem, t1_stack, t1_cost) = dfs(st, coord, vis)
    cost_t1.append(t1_cost)

In [33]:
print "average cost for t1 = " + str(sum(cost_t1) / float(len(cost_t1)))
print "average cost for t2 = " + str(sum(cost_t2) / float(len(cost_t2)))

average cost for t1 = 269.8
average cost for t2 = 233.6


### G3

In [34]:
times_2 = []
for sz in range(3, 21):
    (coord, env, goal, vis, q, st, pq, counter) = initializations(sz,sz,70)
    start_time = timeit.default_timer()
    (path, final_env, t2_nodes, t2_mem, t2_stack, t2_cost) = A_star1(pq, coord, vis, 2)
    t2_time = timeit.default_timer() - start_time
    times_2.append(t2_time)

In [35]:
times_1 = []
for sz in range(3, 21):
    (coord, env, goal, vis, q, st, pq, counter) = initializations(sz,sz,70)
    start_time = timeit.default_timer()
    (path, final_env, t2_nodes, t2_mem, t2_stack, t2_cost) = A_star1(pq, coord, vis, 1)
    t2_time = timeit.default_timer() - start_time
    times_1.append(t2_time)

In [36]:
import matplotlib.pyplot as plt

In [37]:
fig = plt.figure()

In [38]:
ax1 = fig.add_subplot(111)

In [39]:
ax1.scatter(range(3,21), times_1, c='r')

In [40]:
ax1.scatter(range(3, 21), times_2, c='b')

In [41]:
plt.show()

### G4

In [42]:
times_2 = []
for p in range(10, 105, 5):
    (coord, env, goal, vis, q, st, pq, counter) = initializations(10,10,p)
    start_time = timeit.default_timer()
    (path, final_env, t2_nodes, t2_mem, t2_stack, t2_cost) = A_star1(pq, coord, vis, 2)
    t2_time = timeit.default_timer() - start_time
    times_2.append(t2_time)

In [43]:
times_1 = []
for p in range(10,105,5):
    (coord, env, goal, vis, q, st, pq, counter) = initializations(10,10,p)
    start_time = timeit.default_timer()
    (path, final_env, t2_nodes, t2_mem, t2_stack, t2_cost) = A_star1(pq, coord, vis, 1)
    t2_time = timeit.default_timer() - start_time
    times_1.append(t2_time)

In [44]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.scatter(range(10, 105, 5), times_1, c='r')
ax1.scatter(range(10, 105, 5), times_2, c='b')
plt.show()